In [ ]:
!pip3 install scikit-learn
!pip3 install transformers
!pip3 install konlpy
!pip3 install eunjeon
!pip3 install torch
!pip3 install pandas
!pip3 install numpy

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig
import torch
import sklearn
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import os

In [ ]:
malmungchi = pd.read_excel('datas/감성대화말뭉치_훈련용.xlsx', names=['emotion','sentence'])
mmc = malmungchi[['emotion','sentence']]
one_time_conversation = pd.read_excel('datas/단발대화데이터셋_훈련용.xlsx',names=['emotion','sentence'])
otc = one_time_conversation[['emotion','sentence']]
few_times_conversation = pd.read_excel('datas/연속대화데이터셋_훈련용.xlsx',names=['emotion','sentence'])
ftc = few_times_conversation[['emotion','sentence']]

ftc = ftc[ftc['emotion']!='공포']
ftc = ftc[ftc['emotion']!='혐오']
otc = otc[otc['emotion']!='공포']
otc = otc[otc['emotion']!='혐오']

total_data = pd.concat([mmc,otc,ftc])

#결측치제거
total_data = total_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(total_data.isnull().values.any()) # Null 값이 존재하는지 확인
total_data.loc[(total_data['emotion']=='행복'),'emotion'] = 0
total_data.loc[(total_data['emotion']=='기쁨'),'emotion'] = 0
total_data.loc[(total_data['emotion']=='분노'),'emotion'] = 1
total_data.loc[(total_data['emotion']=='놀람'),'emotion'] = 2
total_data.loc[(total_data['emotion']=='당황'),'emotion'] = 2
total_data.loc[(total_data['emotion']=='중립'),'emotion'] = 3
total_data.loc[(total_data['emotion']=='슬픔'),'emotion'] = 4
total_data.loc[(total_data['emotion']=='상처'),'emotion'] = 5
total_data.loc[(total_data['emotion']=='불안'),'emotion'] = 5
total_data = total_data.reset_index(drop=True)
print(total_data)

In [2]:

from konlpy.tag import Mecab
mecab = Mecab(dicpath="/opt/anaconda3/envs/py39/lib/python3.9/site-packages/mecab-ko-dic-2.1.1-20180720")
total_data['sentence'] = total_data['sentence'].map(lambda x: ' '.join(mecab.morphs(x)))

NameError: name 'total_data' is not defined

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(total_data,test_size = 0.2,random_state=123)
train_texts = train_df['sentence'].astype(str).tolist()
train_labels = train_df['emotion'].tolist()
test_texts = test_df['sentence'].astype(str).tolist()
test_labels = test_df['emotion'].tolist()

In [ ]:
model_name = 'monologg/kobert'
tokenizer = BertTokenizer.from_pretrained(model_name)

train_encoding = tokenizer(train_texts,truncation = True, padding = True)
test_encoding = tokenizer(test_texts,truncation = True, padding = True)

In [ ]:
import torch
from torch.utils.data import DataLoader,Dataset

class myDataset(Dataset) : 
    def __init__(self, encodings,labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = myDataset(train_encoding, train_labels)
test_dataset = myDataset(test_encoding, test_labels)
batch_size = 16  # 배치 사이즈는 직접 지정해야 합니다.
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model = BertForSequenceClassification.from_pretrained('monologg/kobert',num_labels=6)

In [ ]:
from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#epoch, lr => hyperparameter(직접 지정해야함)
num_epoches = 10
learning_rate = 2e-5
optimizer = torch.optim.Adam(model.parameters(),lr =learning_rate)
criterion = torch.nn.CrossEntropyLoss()
model.to(device)

for epoch in range(num_epoches):
    model.train() # 훈련 모드 지정
    total_loss = 0

    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epoches} - Average Loss: {average_loss:.4f}")